In [24]:
import pandas as pd
import numpy as np
import glob
import category_encoders as ce
from pydantic import BaseModel
from pandas import DataFrame
# Import the pandas library

# Define the file paths for the datasets
# Get the file paths for all CSV files in the current folder
file_paths = glob.glob('*.csv')

datasets: list[DataFrame] = []
for file_path in file_paths:
    datasets.append(pd.read_csv(file_path))
    


In [25]:
### categorical features with more than 7 unique values will be encode with binary encoding
dataset_one_hot_encoded : list[list[str]] = [[] for i in range(len(datasets))]
dataset_binary_encoded: list[list[str]]   = [[] for i in range(len(datasets))]

In [26]:
## check some information about the dataset
## check if there is any null value
## check the unique counts of each feature in each dataset

for (index,dataset) in enumerate(datasets):
    ### replace ' ' with np.nan and drop examples with na values
    dataset=dataset.replace({' ': np.nan})
    dataset=dataset.replace({'': np.nan})
    dataset=dataset.dropna()
    
    
    print("\n\n")
    print("The unique counts of each feature in this dataset are:")
    print("\n")    
    for column in dataset.columns:
        ### print information about the column
        print(column, dataset[column].dtype, dataset[column].nunique())
        ### cast the column to float if possible ( columns with ' ' has dtype object but now the na are removed we can cast it back to float)
        try:
            dataset[column] = dataset[column].astype(float)
        except ValueError:
            pass
        ### check if the column is categorical and if it has more than 7 unique values pass it to binary encoding, else to one hot encoding    
        if dataset[column].dtype=='object':
            if dataset[column].nunique() > 7:
                dataset_binary_encoded[index].append(column)
            else:
                dataset_one_hot_encoded[index].append(column)
    print("\n\n")
    
    datasets[index] = dataset




The unique counts of each feature in this dataset are:


state object 51
account_length int64 215
area_code object 3
international_plan object 2
voice_mail_plan object 2
number_vmail_messages int64 46
total_day_minutes float64 1843
total_day_calls int64 120
total_day_charge float64 1843
total_eve_minutes float64 1773
total_eve_calls int64 123
total_eve_charge float64 1572
total_night_minutes float64 1757
total_night_calls int64 128
total_night_charge float64 992
total_intl_minutes float64 168
total_intl_calls int64 21
total_intl_charge float64 168
number_customer_service_calls int64 10
churn object 2






The unique counts of each feature in this dataset are:


CustomerID object 1869
Count int64 1
Country object 1
State object 1
City object 833
Zip Code int64 1190
Lat Long object 1190
Latitude float64 1190
Longitude float64 1190
Gender object 2
Senior Citizen object 2
Partner object 2
Dependents object 2
Tenure Months int64 72
Phone Service object 2
Multiple Lines object 3
Internet

In [27]:
for dataset in datasets:
    print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          4250 non-null   object 
 1   account_length                 4250 non-null   float64
 2   area_code                      4250 non-null   object 
 3   international_plan             4250 non-null   object 
 4   voice_mail_plan                4250 non-null   object 
 5   number_vmail_messages          4250 non-null   float64
 6   total_day_minutes              4250 non-null   float64
 7   total_day_calls                4250 non-null   float64
 8   total_day_charge               4250 non-null   float64
 9   total_eve_minutes              4250 non-null   float64
 10  total_eve_calls                4250 non-null   float64
 11  total_eve_charge               4250 non-null   float64
 12  total_night_minutes            4250 non-null   f

In [30]:
from category_encoders import OneHotEncoder, BinaryEncoder

for (index, dataset) in enumerate(datasets):
    for one_hot_column in dataset_one_hot_encoded[index]:
        encoder = OneHotEncoder(cols=[one_hot_column])
        dataset = encoder.fit_transform(dataset)
    for binary_column in dataset_binary_encoded[index]:
        encoder = BinaryEncoder(cols=[binary_column])
        dataset = encoder.fit_transform(dataset)
    datasets[index] = dataset
        
        
        
    

In [31]:
for dataset in datasets:
    print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state_0                        4250 non-null   int64  
 1   state_1                        4250 non-null   int64  
 2   state_2                        4250 non-null   int64  
 3   state_3                        4250 non-null   int64  
 4   state_4                        4250 non-null   int64  
 5   state_5                        4250 non-null   int64  
 6   account_length                 4250 non-null   float64
 7   area_code_1                    4250 non-null   int64  
 8   area_code_2                    4250 non-null   int64  
 9   area_code_3                    4250 non-null   int64  
 10  international_plan_1           4250 non-null   int64  
 11  international_plan_2           4250 non-null   int64  
 12  voice_mail_plan_1              4250 non-null   i

In [33]:
from sklearn.ensemble import RandomForestClassifier

# Assuming 'X' is the input features and 'y' is the target variable
X = datasets[0].drop(['churn_1','churn_2'], axis=1)
y = datasets[0][['churn_1', 'churn_2']]

# Create a Random Forest classifier
rf = RandomForestClassifier()

# Fit the classifier to the data
rf.fit(X, y)

# Get feature importances
feature_importances = rf.feature_importances_

# Create a dictionary of feature importance scores
feature_importance_dict = dict(zip(X.columns, feature_importances))

# Sort the features by importance in descending order
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Set the number of most impactful features to keep
n = 5

# Get the n most impactful features
most_impactful_features = [feature for feature, importance in sorted_features[:n]]

# Print the most impactful features
print("Most impactful features:")
for feature in most_impactful_features:
    print(feature)

KeyError: ('churn_1', 'churn_2')

In [19]:
# ### Turn binary categorical variables into 0 and 1
# pd.set_option('future.no_silent_downcasting', True)
# for dataset in datasets:
#     for column in dataset.columns:
#         if (dataset[column].nunique() == 2) and (dataset[column].dtype == 'object'):
            
#             dataset[column]=dataset[column].replace({'Yes': 'yes', 'No': 'no'})
#             dataset[column]=dataset[column].replace({'yes': 1, 'no': 0})

In [20]:
datasets[0].head()

,state_0,state_1,state_2,state_3,state_4,state_5,account_length,area_code_1,area_code_2,area_code_3,...,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn_1,churn_2
0,0,0,0,0,0,1,107,1,0,0,...,16.62,254.4,103,11.45,13.7,3,3.70,1,1,0
1,0,0,0,0,1,0,137,1,0,0,...,10.30,162.6,104,7.32,12.2,5,3.29,0,1,0
2,0,0,0,0,0,1,84,0,1,0,...,5.26,196.9,89,8.86,6.6,7,1.78,2,1,0
3,0,0,0,0,1,1,75,1,0,0,...,12.61,186.9,121,8.41,10.1,3,2.73,3,1,0
4,0,0,0,1,0,0,121,0,0,1,...,29.62,212.6,118,9.57,7.5,7,2.03,3,1,0


In [21]:
dataset0=datasets[0]

In [22]:
dataset0.nunique()

state_0                             2
state_1                             2
state_2                             2
state_3                             2
state_4                             2
state_5                             2
account_length                    215
area_code_1                         2
area_code_2                         2
area_code_3                         2
international_plan_1                2
international_plan_2                2
voice_mail_plan_1                   2
voice_mail_plan_2                   2
number_vmail_messages              46
total_day_minutes                1843
total_day_calls                   120
total_day_charge                 1843
total_eve_minutes                1773
total_eve_calls                   123
total_eve_charge                 1572
total_night_minutes              1757
total_night_calls                 128
total_night_charge                992
total_intl_minutes                168
total_intl_calls                   21
total_intl_c

In [23]:
encoder = ce.BinaryEncoder(cols=['state'])
df_binary = encoder.fit_transform(dataset0)

ValueError: X does not contain the columns listed in cols

In [ ]:
df_binary.head()

,state_0,state_1,state_2,state_3,state_4,state_5,account_length,area_code,international_plan,voice_mail_plan,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0,0,0,0,0,1,107,area_code_415,0,1,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
1,0,0,0,0,1,0,137,area_code_415,0,0,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
2,0,0,0,0,0,1,84,area_code_408,1,0,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
3,0,0,0,0,1,1,75,area_code_415,1,0,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0
4,0,0,0,1,0,0,121,area_code_510,0,1,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,0
